# Challenge5 不動産サイトからスクレイピング

### 【ミッション】  
● RequestでHTML取得  
● HTMLデータから欲しい情報を取得  
● 取得データをPandasで構造化して保存  

### 【RequestでHTML取得】

>0. 検索する情報の仮決め

不動産サイト：店舗ネットワークから、東京都新宿区の貸店舗・テナント物件一覧  
https://www.temponw.com/area_search/result?prefectures_code=13&name_j=%E6%9D%B1%E4%BA%AC%E9%83%BD&total_groups=2&cities%5B%5D=104

>1. 規約の確認


✔ https://www.temponw.com/robots.txt  
「 Disallow:/rent_store 」を検索 → 無し  
✔ プライバシーポリシー  
スクレイピング・クローリング禁止の記載なし


>2. 環境構築

ターミナルにて、  
 ● pip install requests  
 ● pip install beautifulsoup4  

>3. ライブラリインポート

In [2]:
import requests
from bs4 import BeautifulSoup

>4. htmlデータ取得

In [4]:
REQUESTS_URL = 'https://www.temponw.com/area_search/result?prefectures_code=13&name_j=%E6%9D%B1%E4%BA%AC%E9%83%BD&total_groups=2&cities%5B%5D=104&page=1'
res = requests.get(REQUESTS_URL)

>5. 文字化けチェック

In [5]:
result = res.text
print(result)

<!DOCTYPE html>
<html lang="ja">

<head>
    <!-- Google tag (gtag.js) -->
    <script async src="https://www.googletagmanager.com/gtag/js?id=G-4YFTKLRRJ2"></script>
    <script>
    window.dataLayer = window.dataLayer || [];
    function gtag(){dataLayer.push(arguments);}
    gtag('js', new Date());
    gtag('config', 'G-4YFTKLRRJ2');
    gtag('config', 'UA-209055526-2');
    </script>
    <!-- Google Tag Manager -->
    <script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
    new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
    j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
    'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
    })(window,document,'script','dataLayer','GTM-KDJFPCW');</script>
    <!-- End Google Tag Manager -->
    <meta charset="UTF-8"/>
    <meta name="csrf-token" content="3jgGv5xSYJF2ZGkEN4xXxDF2yuwnOKSzu55MqVpu">
    <meta name="viewport" content="width=device-width, 

>6. 文字コードの確認・変換

In [6]:
# 文字コードの確認
print(res.encoding)

UTF-8


In [7]:
# 文字コードの変換 (→ utf-8 )
res.encoding = 'utf-8'
# 文字化けしていないかhtmlを確認
print(res.text)

<!DOCTYPE html>
<html lang="ja">

<head>
    <!-- Google tag (gtag.js) -->
    <script async src="https://www.googletagmanager.com/gtag/js?id=G-4YFTKLRRJ2"></script>
    <script>
    window.dataLayer = window.dataLayer || [];
    function gtag(){dataLayer.push(arguments);}
    gtag('js', new Date());
    gtag('config', 'G-4YFTKLRRJ2');
    gtag('config', 'UA-209055526-2');
    </script>
    <!-- Google Tag Manager -->
    <script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
    new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
    j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
    'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
    })(window,document,'script','dataLayer','GTM-KDJFPCW');</script>
    <!-- End Google Tag Manager -->
    <meta charset="UTF-8"/>
    <meta name="csrf-token" content="3jgGv5xSYJF2ZGkEN4xXxDF2yuwnOKSzu55MqVpu">
    <meta name="viewport" content="width=device-width, 

### 【HTMLデータから欲しい情報を習得】

>7. BeautifulSoupでhtml解析

In [8]:
# BeautifulSoup(解析したいデータ,解析する方法)を指定し、データをsoupに代入します。
soup = BeautifulSoup(res.text, "html.parser")
soup

<!DOCTYPE html>

<html lang="ja">
<head>
<!-- Google tag (gtag.js) -->
<script async="" src="https://www.googletagmanager.com/gtag/js?id=G-4YFTKLRRJ2"></script>
<script>
    window.dataLayer = window.dataLayer || [];
    function gtag(){dataLayer.push(arguments);}
    gtag('js', new Date());
    gtag('config', 'G-4YFTKLRRJ2');
    gtag('config', 'UA-209055526-2');
    </script>
<!-- Google Tag Manager -->
<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
    new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
    j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
    'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
    })(window,document,'script','dataLayer','GTM-KDJFPCW');</script>
<!-- End Google Tag Manager -->
<meta charset="utf-8"/>
<meta content="3jgGv5xSYJF2ZGkEN4xXxDF2yuwnOKSzu55MqVpu" name="csrf-token"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<link href=

>8. 欲しい情報を1つずつしか持たないhtmlを抽出する

● HP上で右クリック  
● 検証　を選択 → HTMLコードが見れるようになる  
● 「ページ上の検査対象となる要素の選択」ボタンをクリック  


●　物件名：label.checkbox-container    
●　所在地：div.result-detail-row  
●　最寄駅：div.result-detail-row  
●　賃料：div.result-detail-row.border-top-line  
●　管理費：div.result-detail-row  
●　面積：div.result-detail-row  
●　使用階：div.result-detail-row  
●　用途：div.detail-grey  

![Alt text](image-1.png)

In [60]:
# すべてのsectionタグを取得します。
property_section = soup.select('div.result-contents.sm-hidden')
# 取得したセクションすべての数を確認します。
len(property_section)

10

>9. 1つの物件の欲しい情報を取得

In [84]:
ps = property_section[0]
# 物件名
print(ps.select_one('label.checkbox-container').text)
# 所在地
print(ps.select('div.result-detail-row')[1].text)
# 最寄駅
print(ps.select('div.result-detail-row')[2].text)
# 賃料
print(ps.select('div.result-detail-row.border-top-line')[0].text)
# 面積
print(ps.select('div.result-detail-row')[5].text)
# 店舗特徴
print(ps.select('div.detail-grey'))



西早稲田１階店舗


所在地
東京都新宿区西早稲田1-23-13


最寄駅
都電荒川線　早稲田、他...                


賃料
月額 41.8万円 税込


面積

                                                                    80.44 m2（24.33 坪）
                                                            

[<div class="detail-grey">軽飲食</div>, <div class="detail-grey">物販</div>, <div class="detail-grey">理美容・サロン</div>, <div class="detail-grey">クリニック・介護</div>, <div class="detail-grey">アミューズメント</div>, <div class="detail-grey">事務所</div>, <div class="detail-grey">その他</div>, <div class="detail-grey">天高2.5ｍ以上</div>, <div class="detail-grey">スケルトン</div>, <div class="detail-grey">駅前</div>, <div class="detail-grey">路面店（1階）</div>]


>10. データ整理

In [87]:
# 所在地(記載整理)
ps.select('div.result-detail-row')[1].text.replace("\n所在地\n","").replace("\n","")

'東京都新宿区西早稲田1-23-13'

In [93]:
# 最寄駅(記載整理)
ps.select('div.result-detail-row')[2].text.replace("\n最寄駅\n","").replace("\n","").replace(" ","").replace("\u3000"," ").replace("、他...","")

'都電荒川線 早稲田'